In [46]:
import fastai.vision.all as fai
import fastbook as faibook

fai.matplotlib.rc('image', cmap='Greys')

In [47]:
# let's get images and transform them 
# to data with which we can work in NN, like this:
# Image -> [[0,0,255]] -> [[0.0, 0.0, 1.0]]

In [48]:
path_to_imgs = fai.untar_data(fai.URLs.MNIST_SAMPLE)
path_to_imgs

Path('/Users/mxz/.fastai/data/mnist_sample')

In [49]:
# we have some directories and files here
path_to_imgs.ls()

(#3) [Path('/Users/mxz/.fastai/data/mnist_sample/valid'),Path('/Users/mxz/.fastai/data/mnist_sample/labels.csv'),Path('/Users/mxz/.fastai/data/mnist_sample/train')]

In [50]:
train_threes_path = (path_to_imgs/'train'/'3').ls().sorted()
train_sevens_path = (path_to_imgs/'train'/'7').ls().sorted()

train_threes_path[0]

Path('/Users/mxz/.fastai/data/mnist_sample/train/3/10.png')

In [51]:
example_img = fai.Image.open(train_threes_path[1])
example_img

In [52]:
# to view the numbers that make up this image, 
# we have to convert it to a NumPy array or a PyTorch tensor
example_img_as_numbers = fai.tensor(example_img)

# 255 -> more "bright", dark, 0 -> more transparent, white
# we can visualize those numbers, uncomment code below for it
# df = fai.pd.DataFrame(example_img_as_numbers)
# df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')


In [53]:
three_tensors = [fai.tensor(fai.Image.open(img)) for img in train_threes_path]
seven_tensors = [fai.tensor(fai.Image.open(img)) for img in train_sevens_path]

stacked_sevens = fai.torch.stack(seven_tensors).float() / 255
stacked_threes = fai.torch.stack(three_tensors).float() / 255

print(stacked_threes.size())
print(stacked_sevens.size())

torch.Size([6131, 28, 28])
torch.Size([6265, 28, 28])


In [54]:
img_width = img_height = 28

In [58]:
all_img_sample_tensors = fai.torch.cat([stacked_threes, stacked_sevens])
all_img_sample_tensors.size()

torch.Size([12396, 28, 28])

In [61]:
samples_len = all_img_sample_tensors.size()[0]
samples_len

12396

In [68]:
# TODO: why we want to do it?

# We want to change samples from a list of matrices (a rank-3 tensor) to a list of vectors (a rank-2 tensor). 
# We can do this using `view`, which is a PyTorch method that changes the shape of a tensor without changing its contents. 

In [62]:
train_x = all_img_sample_tensors.view(samples_len, img_width * img_height)
# instead of samples_len we can use -1 to ask torch tensor to calclulate size for us
# "make this axis as big as necessary to fit all the data"
# all_img_sample_tensors.view(-1, img_width * img_height)
train_x.size()

torch.Size([12396, 784])